In [ ]:
import os

from PIL import Image
from torchvision import transforms
import numpy as np

In [2]:
transform = transforms.Compose([
    transforms.Resize((6,6)),
    transforms.ToTensor(),
])

In [3]:
images = [Image.open(os.path.join('samples', f'{i}.png')) for i in range(10)]

In [4]:
# load model parameters for cpu

model = np.load('../quantized_model_brevitas.npz')

W1_q = model['W1_q']
b1_q = model['b1_q']
M_fc1 = model['M_fc1']
S_fc1 = model['S_fc1']
scale_fc1_in = model['scale_fc1_in']

W2_q = model['W2_q']
b2_q = model['b2_q']
M_fc2 = model['M_fc2']
S_fc2 = model['S_fc2']

In [ ]:
def int8_matvec(W_q: np.ndarray, a_q: np.ndarray, bias_q: np.ndarray):
    acc = W_q.astype(np.int32).dot(a_q.astype(np.int32))
    if bias_q is not None:
        acc = acc + bias_q.astype(np.int32)
    return acc

def requantize_accumulator(acc: np.ndarray, M: np.ndarray, S: np.ndarray, out_dtype=np.int8):
    acc64 = acc.astype(np.int64) * M.astype(np.int64)
    res = (acc64 >> S.astype(np.int64)).astype(out_dtype)
    return res

def quantize_input_image(x):
    x_flat = (x.flatten() / scale_fc1_in)
    x_q = np.round(x_flat).astype(np.int8)
    return x_q

def inference_numpy_cpu(x):
    x_q = quantize_input_image(x)
    a0 = x_q.astype(np.int32)
    acc1 = int8_matvec(W1_q, a0, b1_q)
    r1 = requantize_accumulator(acc1, M_fc1, S_fc1, out_dtype=np.int8)
    r1 = r1.astype(np.int32)
    r1[r1 < 0] = 0
    acc2 = int8_matvec(W2_q, r1.astype(np.int8), b2_q)
    acc2_shared = requantize_accumulator(acc2, M_fc2, S_fc2, out_dtype=np.int32)
    print(acc2_shared)
    pred = int(np.argmax(acc2_shared))
    return pred

def inference_cpu(x):
    x = transform(x).numpy()
    return inference_numpy_cpu(x)

In [ ]:
from pynq import Overlay, GPIO

ps_gpio_design = Overlay("./design_1.bit")

READ_ADDR  = 34
WRITE_ADDR = 18
DATA       = 10
VALID      =  9
RESULT     =  5
DONE       =  4
MAGIC      =  0

ModuleNotFoundError: No module named 'pynq'

In [34]:
gpios = []

# magic
for i in range(MAGIC,DONE):
    gpios.append(GPIO(GPIO.get_gpio_pin(i), 'out'))

# done
for i in range(DONE,RESULT):
    gpios.append(GPIO(GPIO.get_gpio_pin(i), 'out'))

# result
for i in range(RESULT,DATA):
    gpios.append(GPIO(GPIO.get_gpio_pin(i), 'out'))

# data
for i in range(DATA, WRITE_ADDR):
    gpios.append(GPIO(GPIO.get_gpio_pin(i), 'in'))

# write addr
for i in range(WRITE_ADDR, READ_ADDR):
    gpios.append(GPIO(GPIO.get_gpio_pin(i), 'in'))

# read addr
for i in range(READ_ADDR, 50):
    gpios.append(GPIO(GPIO.get_gpio_pin(i), 'out'))

def read_bit(off):
    return gpios[off].read()

def write_bit(off, val):
    return gpios[off].write(val)

def read_uint(off, size):
    val = 0
    for i in range(size):
        val |= read_bit(off + i) << i
    return val

def write_int(off, size, val):
    for i in range(size):
        gpios[i].write(val & (1 << i))

def read_magic():
    return read_uint(MAGIC, DONE - MAGIC)

def read_done():
    return read_bit(DONE)

def read_result():
    return read_uint(RESULT, VALID - RESULT)

def write_valid(val):
    write_bit(VALID, val)

def write_data(val):
    write_int(DATA, WRITE_ADDR - DATA, val)
    
def write_addr(val):
    write_int(WRITE_ADDR, READ_ADDR - WRITE_ADDR, val)

def read_addr():
    return read_uint(READ_ADDR, 50 - READ_ADDR)

def load_input(x):
    # wait until PL requests first input
    while read_addr(i) != 0: pass
    write_valid(1)

    for i, xi in enumerate(x):
        write_addr(i)
        write_data(xi)
        # wait until PL can read next input
        while read_addr(i) != i + 1: pass
    write_valid(0)

def inference_fpga(x):
    x = transform(x).numpy()

    # quantize input on CPU (float32 -> int8)
    x = quantize_input_image(x)

    # load the input into the PL
    load_input(x)

    print("Loaded the input.")

    # wait done
    while read_done() == 0: pass
    return read_result()

print(f'The magic number is {read_magic()}')

NameError: name 'MAGIC' is not defined

In [ ]:
images[0]

In [ ]:
inference_cpu(images[0])


[  8411 -17183   1979   1584  -3652   1834   2896   -484   1638   1661]


0

In [ ]:
inference_fpga(images[0])